In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
def add_value_labels(ax, spacing=5):
    """Add labels to the end of each bar in a bar chart.

    Arguments:
        ax (matplotlib.axes.Axes): The matplotlib object containing the axes
            of the plot to annotate.
        spacing (int): The distance between the labels and the bars.
    """

    # For each bar: Place a label
    for rect in ax.patches:
        # Get X and Y placement of label from rect.
        y_value = rect.get_height()
        x_value = rect.get_x() + rect.get_width() / 2

        # Number of points between bar and label. Change to your liking.
        space = spacing
        # Vertical alignment for positive values
        va = 'bottom'

        # If value of bar is negative: Place label below bar
        if y_value < 0:
            # Invert space to place label below
            space *= -1
            # Vertically align label at top
            va = 'top'

        # Use Y value as label and format number with one decimal place
        label = "{:.0f}".format(y_value)

        # Create annotation
        ax.annotate(
            label,                      # Use `label` as label
            (x_value, y_value),         # Place label at end of the bar
            xytext=(0, space),          # Vertically shift label by `space`
            textcoords="offset points", # Interpret `xytext` as offset in points
            ha='center',                # Horizontally center label
            va=va)                      # Vertically align label differently for
                                        # positive and negative values.


In [3]:
fpath = 'family_data.csv'
data = pd.read_csv(fpath, index_col='family_id')

fpath = 'sample_submission.csv'
submission = pd.read_csv(fpath, index_col='family_id')

In [4]:
data.head(2)

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4


In [23]:
data.columns

Index(['choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5',
       'choice_6', 'choice_7', 'choice_8', 'choice_9', 'n_people'],
      dtype='object')

In [26]:
pd.melt(data, value_vars=['choice_0', 'choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5',
       'choice_6', 'choice_7', 'choice_8', 'choice_9'],value_name='preferred_day')

,variable,preferred_day
0,choice_0,52
1,choice_0,26
2,choice_0,100
3,choice_0,2
4,choice_0,53
...,...,...
49995,choice_9,60
49996,choice_9,48
49997,choice_9,7
49998,choice_9,70


In [5]:
submission.head(2)

,assigned_day
family_id,
0,100
1,99


In [17]:
# submission['n_people'] = data['n_people']
submission['preference'] = np.nan
submission.T

family_id,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
assigned_day,100.0,99.0,98.0,97.0,96.0,95.0,94.0,93.0,92.0,91.0,...,10.0,9.0,8.0,7.0,6.0,5.0,4.0,3.0,2.0,1.0
preference,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
data.T

family_id,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
choice_0,52,26,100,2,53,32,88,25,18,1,...,47,45,12,62,52,16,88,32,67,13
choice_1,38,4,54,95,1,59,4,11,60,88,...,61,47,75,1,59,1,66,66,92,11
choice_2,12,82,25,1,47,12,1,52,1,39,...,74,29,31,89,1,66,20,54,4,25
choice_3,82,5,12,96,93,3,3,48,12,50,...,12,1,90,66,54,33,17,17,17,80
choice_4,33,11,27,32,26,60,91,10,89,26,...,80,83,22,28,80,18,26,27,53,88
choice_5,75,47,82,6,3,26,32,17,33,18,...,46,5,26,60,87,70,54,21,77,40
choice_6,64,38,10,40,46,35,39,88,16,96,...,32,49,76,81,16,56,81,74,1,96
choice_7,76,6,89,31,16,50,57,50,10,47,...,94,14,32,40,47,46,91,81,12,39
choice_8,10,66,80,9,42,5,28,95,53,46,...,4,91,1,88,13,86,59,3,26,18
choice_9,28,61,33,59,39,2,99,66,67,28,...,19,75,52,13,15,60,48,7,70,47


In [16]:
for index,row in data.T.iterrows():
    print(row)

family_id
0        52
1        26
2       100
3         2
4        53
       ... 
4995     16
4996     88
4997     32
4998     67
4999     13
Name: choice_0, Length: 5000, dtype: int64
family_id
0       38
1        4
2       54
3       95
4        1
        ..
4995     1
4996    66
4997    66
4998    92
4999    11
Name: choice_1, Length: 5000, dtype: int64
family_id
0       12
1       82
2       25
3        1
4       47
        ..
4995    66
4996    20
4997    54
4998     4
4999    25
Name: choice_2, Length: 5000, dtype: int64
family_id
0       82
1        5
2       12
3       96
4       93
        ..
4995    33
4996    17
4997    17
4998    17
4999    80
Name: choice_3, Length: 5000, dtype: int64
family_id
0       33
1       11
2       27
3       32
4       26
        ..
4995    18
4996    26
4997    27
4998    53
4999    88
Name: choice_4, Length: 5000, dtype: int64
family_id
0       75
1       47
2       82
3        6
4        3
        ..
4995    70
4996    54
4997    21
4998    77

In [9]:
# Given cost function for families not getting their desired preferences
def top_pref_penalty(pref,d):
    
    penalty = 0
    n = pref
    
    d = d
    if pref > 0:
        if d == 0:
            penalty += 0
        elif d == 1:
            penalty += 50
        elif d == 2:
            penalty += 50 + 9 * n
        elif d == 3:
            penalty += 100 + 9 * n
        elif d == 4:
            penalty += 200 + 9 * n
        elif d == 5:
            penalty += 200 + 18 * n
        elif d == 6:
            penalty += 300 + 18 * n
        elif d == 7:
            penalty += 300 + 36 * n
        elif d == 8:
            penalty += 400 + 36 * n
        elif d == 9:
            penalty += 500 + 36 * n + 199 * n
        else:
            penalty += 500 + 36 * n + 398 * n
    else:
        pass
        
    return penalty

In [11]:
submission[['assigned_day','n_people']].groupby('assigned_day').sum()

KeyError: "['n_people'] not in index"